In [2]:
from pathlib import Path
import json
import pandas as pd


# 加载最新数据集
def load_latest_dataset(
    symbol: str,
    timeframe: str,
    exchange: str = "binance",
    data_dir: str = "../data/raw",
) -> tuple[pd.DataFrame, dict, Path]:
    symbol_clean = symbol.replace("/", "_").lower()
    pattern = f"{exchange}_{symbol_clean}_{timeframe}.csv"

    # 查找所有匹配文件
    all_files = sorted(Path(data_dir).rglob(pattern), reverse=True)

    if not all_files:
        raise FileNotFoundError(f"未找到匹配文件: {pattern}")

    latest_file = all_files[0]
    meta_file = latest_file.with_name(latest_file.name + ".meta.json")

    # 加载数据
    df = pd.read_csv(
        latest_file,
        parse_dates=["datetime"],
        index_col="datetime",
        dtype={
            "open": float,
            "high": float,
            "low": float,
            "close": float,
            "volume": float,
        },
    )

    # 加载元数据
    with open(meta_file, "r") as f:
        metadata = json.load(f)

    # 数据校验
    current_hash = pd.util.hash_pandas_object(df).sum()
    if current_hash != metadata["data_hash"]:
        raise ValueError("数据校验失败！文件可能已被修改")

    return df, metadata, latest_file


df, meta, latest_file = load_latest_dataset(
    symbol="BTC/USDT",
    timeframe="15m",
    exchange="binance",
    data_dir="../../data/raw",
)

print(f"成功加载数据：{latest_file.name}")
print(f"数据范围：{df.index.min()} ~ {df.index.max()}")

成功加载数据：binance_btc_usdt_15m.csv
数据范围：2025-01-23 17:30:00+00:00 ~ 2025-01-30 17:15:00+00:00


In [3]:
def diagnose_data(df: pd.DataFrame, timeframe: str) -> dict:
    """综合数据健康诊断报告"""
    report = {}

    # 时间连续性检查
    time_diff = df.index.to_series().diff().dropna()
    expected_interval = pd.Timedelta(timeframe)
    gaps = time_diff[time_diff > expected_interval]
    report["time_continuity"] = {
        "missing_intervals": gaps.shape[0],
        "max_gap": gaps.max().total_seconds() / 60 if not gaps.empty else 0,
        "expected_interval_min": expected_interval.total_seconds() / 60,
    }

    # 价格合理性检查
    price_checks = {
        "zero_prices": df[["open", "high", "low", "close"]].eq(0).any().any(),
        "high_low_order": (df["high"] < df["low"]).sum(),
        "open_close_range": (
            (df["close"] > df["high"]) | (df["close"] < df["low"])
        ).sum(),
    }
    report["price_sanity"] = price_checks

    # 成交量异常检测
    volume_outliers = df["volume"][df["volume"] < 0]
    report["volume_issues"] = {
        "negative_volume": volume_outliers.count(),
        "volume_skewness": df["volume"].skew(),
    }

    return report

# 使用示例
diagnosis = diagnose_data(df, meta["params"]["timeframe"])
print(json.dumps(diagnosis, indent=2, default=str))

{
  "time_continuity": {
    "missing_intervals": 0,
    "max_gap": 0,
    "expected_interval_min": 15.0
  },
  "price_sanity": {
    "zero_prices": "False",
    "high_low_order": "0",
    "open_close_range": "0"
  },
  "volume_issues": {
    "negative_volume": "0",
    "volume_skewness": 4.03075184419423
  }
}


In [4]:
import numpy as np


# 价差波动率Z值阈值 (建议范围2.5-3.5)
Z_THRESHOLD = 3.0
    
# 滚动窗口周期（按K线数量计）
# 假设15分钟线：24h*7天 = 24*4=96条/天 → 96*7=672条
ROLLING_WINDOW = 24 * 7  # 保持与原始代码一致
    
# 价格突变检测倍数
PRICE_CHANGE_SIGMA = 3  # 标准差倍数

# 最小有效价格（防止除零错误）
MIN_PRICE = 0.01  # 当low低于此值时视为无效数据

def detect_anomalies(
    df: pd.DataFrame,
    z_threshold: float = Z_THRESHOLD,
    price_sigma: float = PRICE_CHANGE_SIGMA,
) -> pd.DataFrame:
    """基于波动率的异常K线检测"""

    # 计算价格变化率（带价格有效性检查）
    df = df.assign(
        price_change=df["close"].pct_change().abs(),
        spread_ratio=(
            (df["high"] - df["low"])
            / df["low"].clip(lower=MIN_PRICE)  # 防止low接近零时出现极端值
        ),
    )

    # 动态Z-Score计算
    rolling_mean = (
        df["spread_ratio"]
        .rolling(
            ROLLING_WINDOW,
            min_periods=int(ROLLING_WINDOW * 0.1),  # 允许最小10%数据量
        )
        .mean()
    )

    rolling_std = (
        df["spread_ratio"]
        .rolling(
            ROLLING_WINDOW,
            min_periods=int(ROLLING_WINDOW * 0.1),
        )
        .std()
    )

    df["spread_z"] = (df["spread_ratio"] - rolling_mean) / rolling_std

    # 标记异常（使用可配置参数）
    price_threshold = price_sigma * df["price_change"].std()
    df["is_anomaly"] = np.where(
        (df["spread_z"] > z_threshold) | (df["price_change"] > price_threshold), 1, 0
    )

    return df


# 可视化异常点
anomaly_df = detect_anomalies(df)
anomaly_df[anomaly_df["is_anomaly"] == 1].index

DatetimeIndex(['2025-01-23 18:00:00+00:00', '2025-01-23 19:45:00+00:00',
               '2025-01-23 20:00:00+00:00', '2025-01-23 20:15:00+00:00',
               '2025-01-23 20:30:00+00:00', '2025-01-23 20:45:00+00:00',
               '2025-01-24 14:30:00+00:00', '2025-01-26 22:45:00+00:00',
               '2025-01-26 23:00:00+00:00', '2025-01-26 23:15:00+00:00',
               '2025-01-26 23:45:00+00:00', '2025-01-27 00:00:00+00:00',
               '2025-01-27 00:45:00+00:00', '2025-01-27 01:00:00+00:00',
               '2025-01-27 01:15:00+00:00', '2025-01-27 02:00:00+00:00',
               '2025-01-27 03:00:00+00:00', '2025-01-27 06:30:00+00:00',
               '2025-01-27 07:15:00+00:00', '2025-01-27 07:30:00+00:00',
               '2025-01-27 12:30:00+00:00', '2025-01-27 12:45:00+00:00',
               '2025-01-27 13:00:00+00:00', '2025-01-27 13:15:00+00:00',
               '2025-01-27 14:15:00+00:00', '2025-01-27 14:45:00+00:00',
               '2025-01-27 16:30:00+00:00', '2025-0

In [9]:
from typing import Dict, Optional

# 配置常量（替代类的配置）
DEFAULT_CLEAN_CONFIG = {
    "fill_method": "time",  # 支持 time/index 等更专业的填充方式
    "max_price_jump_ratio": 0.15,
    "timeframe": "15T",  # 改用 pandas 标准频率字符串
    "min_volume": 0.01,  # 新增过滤低成交
}


def create_clean_pipeline(config: Optional[Dict] = None):
    """工厂函数生成清洗流程（替代类构造函数）"""
    final_config = {**DEFAULT_CLEAN_CONFIG, **(config or {})}

    # 闭包函数保持配置状态
    def _handle_gaps(df: pd.DataFrame) -> pd.DataFrame:
        """更稳健的时间缺口处理"""
        full_index = pd.date_range(
            start=df.index.min().floor(final_config["timeframe"]),
            end=df.index.max().ceil(final_config["timeframe"]),
            freq=final_config["timeframe"],
        )
        return (
            df.reindex(full_index)
            .assign(is_gap=lambda x: x["close"].isna())
            .interpolate(method=final_config["fill_method"])
        )

    def _filter_anomalies(df: pd.DataFrame) -> pd.DataFrame:
        """更科学的异常过滤（修正原逻辑错误）"""
        return df[
            (df["high"] / df["low"] - 1 < final_config["max_price_jump_ratio"])
            & (df["volume"] >= final_config["min_volume"])  # 新增成交量过滤
        ].copy()

    # 返回处理函数链
    return [_handle_gaps, detect_anomalies, _filter_anomalies]  # 假设已有该函数


# 使用示例
clean_steps = create_clean_pipeline({"timeframe": "5min", "max_price_jump_ratio": 0.2})

# 链式执行（更符合 Notebook 的探索特性）
cleaned_data = df.pipe(clean_steps[0]).pipe(clean_steps[1]).pipe(clean_steps[2])
cleaned_data

,timestamp,open,high,low,close,volume,is_gap,price_change,spread_ratio,spread_z,is_anomaly
2025-01-23 17:30:00+00:00,1.737653e+12,105286.940000,105680.000000,105224.130000,105591.680000,450.043380,False,NaN,0.004332,NaN,0
2025-01-23 17:35:00+00:00,1.737654e+12,105388.520000,105863.473333,105334.486667,105703.713333,454.776907,True,0.001061,0.005022,NaN,0
2025-01-23 17:40:00+00:00,1.737654e+12,105490.100000,106046.946667,105444.843333,105815.746667,459.510433,True,0.001060,0.005710,NaN,0
2025-01-23 17:45:00+00:00,1.737654e+12,105591.680000,106230.420000,105555.200000,105927.780000,464.243960,False,0.001059,0.006397,NaN,0
2025-01-23 17:50:00+00:00,1.737655e+12,105703.713333,106201.100000,105426.953333,105690.956667,424.426563,True,0.002236,0.007343,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-30 16:55:00+00:00,1.738256e+12,105729.866667,105746.740000,105389.990000,105455.926667,214.952517,True,0.001142,0.003385,0.249048,0
2025-01-30 17:00:00+00:00,1.738256e+12,105696.960000,105698.270000,105272.720000,105335.410000,253.255010,False,0.001143,0.004042,0.641213,0
2025-01-30 17:05:00+00:00,1.738257e+12,105576.440000,105597.310000,105243.146667,105349.696667,190.473410,True,0.000136,0.003365,0.226861,0
2025-01-30 17:10:00+00:00,1.738257e+12,105455.920000,105496.350000,105213.573333,105363.983333,127.691810,True,0.000136,0.002688,-0.186789,0
